In [ ]:

import os

import glob


import numpy as np 

import pandas as pd 


import matplotlib.pyplot as plt

import seaborn as sns


import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.layers import Activation, ZeroPadding3D, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
import tensorflow as tf

from tensorflow.keras.utils import to_categorical, plot_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# Data

### List files

In [ ]:

os.listdir('../input/cell-images-for-detecting-malaria/cell_images/cell_images')

In [ ]:

parasitized = glob.glob('../input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized/*.png')

print('No. of files in the directory Parasitized', len(parasitized))


parasitized[:5]

In [ ]:
# list all the images in the directory Uninfected
uninfected = glob.glob('../input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected/*.png')

# no. of files in the directory Uninfected
print('No. of files in the directory Uninfected', len(uninfected))

# first few images
uninfected[:5]

# Images

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
fig.suptitle('Infected cells', fontsize=24)

for Indx, ImgSrc in enumerate(parasitized[:30]):
    plt.subplot(3, 10, Indx+1)
    imge = plt.imread(ImgSrc)
    plt.axis('off')
    plt.imshow(imge)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
fig.suptitle('Uninfected cells', fontsize=24)

for Indx, ImgSrc in enumerate(uninfected[:30]):
    plt.subplot(3, 10, Indx+1)
    imge = plt.imread(ImgSrc)
    plt.axis('off')
    plt.imshow(imge)

# Model

### Model parameters

In [ ]:
BATCH_SIZE = 100  
IMG_SHAPE  = 150 
TARGET_SIZE = 64
EPOCHS = 1

### Model initialization

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SHAPE, IMG_SHAPE, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())

model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
plt.figure(figsize=(5, 10))
plot_model(model, to_file="model.png")

### Data generator

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             validation_split=0.3)

train_data = datagen.flow_from_directory('../input/cell-images-for-detecting-malaria/cell_images/cell_images',
                                         target_size=(IMG_SHAPE,IMG_SHAPE),
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         class_mode='binary',
                                         subset='training')

validation_data = datagen.flow_from_directory('../input/cell-images-for-detecting-malaria/cell_images/cell_images',
                                              target_size=(IMG_SHAPE,IMG_SHAPE),
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              class_mode='binary',
                                              subset='validation')

### Callback functions

In [ ]:
# Instantiate an early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', 
                               min_delta = 0.01,
                               patience=5)

# Instantiate a model checkpoint callback
model_save = ModelCheckpoint('best_model_custom.hdf5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True)

### Fit model

In [ ]:
history = model.fit(train_data,
                    validation_data=validation_data,
                    epochs=EPOCHS,
                    verbose=1, 
                    callbacks=[early_stopping, model_save])

### Plot metrics

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()